In [18]:
import json
pred_data_1=[]
pred_num_1=0
with open("./predict_res kg.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data_1.append(json.loads(d))
        pred_num_1+=len(pred_data_1[-1]['spo_list'])
pred_num_1

23861

In [11]:
import unicodedata
def _is_whitespace(char):
    """Checks whether `chars` is a whitespace character."""
    # \t, \n, and \r are technically contorl characters but we treat them
    # as whitespace since they are generally considered as such.
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False
def is_chinese_char(cp):
    """Checks whether CP is the codepoint of a CJK character."""
    # This defines a "chinese character" as anything in the CJK Unicode block:
    #   https://en.wikipedia.org/wiki/CJK_Unified_Ideographs_(Unicode_block)
    #
    # Note that the CJK Unicode block is NOT all Japanese and Korean characters,
    # despite its name. The modern Korean Hangul alphabet is a different block,
    # as is Japanese Hiragana and Katakana. Those alphabets are used to write
    # space-separated words, so they are not treated specially and handled
    # like the all of the other languages.
    if (
        (cp >= 0x4E00 and cp <= 0x9FFF)
        or (cp >= 0x3400 and cp <= 0x4DBF)  #
        or (cp >= 0x20000 and cp <= 0x2A6DF)  #
        or (cp >= 0x2A700 and cp <= 0x2B73F)  #
        or (cp >= 0x2B740 and cp <= 0x2B81F)  #
        or (cp >= 0x2B820 and cp <= 0x2CEAF)  #
        or (cp >= 0xF900 and cp <= 0xFAFF)
        or (cp >= 0x2F800 and cp <= 0x2FA1F)  #
    ):  #
        return True

    return False
def is_punctuation(char):
    """Checks whether `chars` is a punctuation character."""
    cp = ord(char)
    # We treat all non-letter/number ASCII as punctuation.
    # Characters such as "^", "$", and "`" are not in the Unicode
    # Punctuation class but we treat them as punctuation anyways, for
    # consistency.
    if (cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [12]:
for idx,e in enumerate(pred_data_1):
    for spo in e['spo_list']:
        while spo['subject'][0]=='《' and ( spo['subject'][-1]=='》' or  spo['subject'].find("》")==-1):
            print("sub",spo['subject'])
            spo['subject']=spo['subject'].replace("《","").replace("》","")
            print("sub",spo['subject'])         
            break
        poped_keys=[]
        for key,value in spo['object'].items():
            if len(value)==0:
                print(spo)
                poped_keys.append(key)
            if len(poped_keys)>0:
                for t in poped_keys:
                    print(t)
                    spo['object'].pop(t)
                    spo['object_type'].pop(t)
                print(idx,pred_data_1[idx])
                break
            while value[0]=='《' and (value[-1]=='》' or value.find("》")==-1):
                print("ob",key,value)
                spo['object'][key]=value.replace("《","").replace("》","")
                print("ob",spo['object'][key])        
                break

In [13]:
with open("./predict_res kg.json","w") as w:
    for e in pred_data_1:
        w.write(json.dumps(e, ensure_ascii=False)+'\n')


In [15]:
pred_data_2=[]
pred_num_2=0
with open("./frzhu/new_test1_res kg.json","r") as r:
    raw_data=r.readlines()
    for d in raw_data:
        pred_data_2.append(json.loads(d))
        pred_num_2+=len(pred_data_2[-1]['spo_list'])
pred_num_2

17964

In [16]:
def cross_contain(a,b):
    if a.find(b)!=-1 or b.find(a)!=-1:
        return True
    return False
for idx in range(len(pred_data_1)):
    for spo in pred_data_2[idx]['spo_list']:
        flag=True
        for temp in pred_data_1[idx]['spo_list']:
            if temp['subject']==spo['subject'] and temp['predicate']==spo['predicate']  and cross_contain(temp['object']['@value'],spo['object']['@value'])\
               and len(temp['object']['@value'])>len(spo['object']['@value']) and len(spo['object']['@value'])<4:
                flag=False
                temp['object']['@value']=spo['object']['@value']
                break
#         if flag:
#             pred_data_1[idx]['spo_list'].append(spo)
# 7
# 9

In [17]:
with open("./predict_res kg combine.json","w") as w:
    for e in pred_data_1:
        w.write(json.dumps(e, ensure_ascii=False)+'\n')
